This module performs performs conversions from csv to pandas dataframes

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Define-all-functions-within-notebook" data-toc-modified-id="Define-all-functions-within-notebook-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define all functions within notebook</a></span></li><li><span><a href="#Define-all-functions" data-toc-modified-id="Define-all-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define all functions</a></span><ul class="toc-item"><li><span><a href="#FUNCTION---csv_to_df" data-toc-modified-id="FUNCTION---csv_to_df-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>FUNCTION - csv_to_df</a></span></li><li><span><a href="#FUNCTION---head_csv_to_df" data-toc-modified-id="FUNCTION---head_csv_to_df-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>FUNCTION - head_csv_to_df</a></span></li><li><span><a href="#FUNCTION---merge_files_on_col" data-toc-modified-id="FUNCTION---merge_files_on_col-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>FUNCTION - merge_files_on_col</a></span></li><li><span><a href="#FUNCTION---stack_csvs_into_df" data-toc-modified-id="FUNCTION---stack_csvs_into_df-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>FUNCTION - stack_csvs_into_df</a></span></li></ul></li></ul></div>

# Imports

In [47]:
import pandas as pd

# Define all functions within notebook

In [16]:
def in_out_def():
    return {
        'csv_to_df':{'inputs':{"file":"file_browse"},'outputs':{"df":"variable"}},
        'head_csv_to_df':{'inputs':{"file":"file_browse","header":"int_input"},'outputs':{"df":"variable"}},
        'merge_files_on_col':{'inputs':{'df':"variable","new_df":"variable","new_merge_col":"text_input",
                                        "master_merge_col":"text_input","new_other_cols":"text_input"}
                              ,'outputs':{"df":"variable"}},
        'stack_csvs_into_df':{'inputs':{'files':'files_browse','header_row':"int_input"},'outputs':{"df":"variable"}}
    }

# Define all functions

## FUNCTION - csv_to_df
Simple csv to df convert usign default settings

In [3]:
def csv_to_df(file=None):
    '''Take csv file and convert to dataframe'''
    if file != None:
        try:
            return {"df":pd.read_csv(file)}
        except:
            pass
    return {"df":None}

## FUNCTION - head_csv_to_df

In [4]:
def head_csv_to_df(file=None,header=None):
    '''Take csv file and convert to dataframe'''
    if file != None and header!=None:
        try:
            return {"df":pd.read_csv(file,header=header)}
        except:
            pass
    return {"df":None}

## FUNCTION - merge_files_on_col

In [2]:
def merge_files_on_col(df=None,new_df=None,new_merge_col=None,master_merge_col=None,
                       new_other_cols=None):
    '''Take two dfs and merge on specified columns, keeping specified columns of second df'''
    if isinstance(df,pd.DataFrame) and isinstance(new_df,pd.DataFrame) and None not in [new_merge_col,master_merge_col,new_other_cols]:
        if (not df.empty) and (not new_df.empty):
            #try:
            if not isinstance(new_other_cols,list):
                    new_other_cols="".join(new_other_cols.split()).split(',')
            print(df,new_df,new_merge_col,master_merge_col,new_other_cols)
            to_merge=new_df[[new_merge_col]+new_other_cols].copy()
            to_merge[new_merge_col]=to_merge[new_merge_col].apply(lambda val:''.join(e.upper() for e in str(val) if e.isalnum()))
            df[master_merge_col]=df[master_merge_col].apply(lambda val:''.join(e.upper() for e in str(val) if e.isalnum()))
            to_merge=to_merge.drop_duplicates(subset=new_merge_col,keep='last')
            to_merge=to_merge.dropna()
            df=pd.merge(df,to_merge,left_on=master_merge_col,right_on=new_merge_col,how='outer')
            if master_merge_col!=new_merge_col:
                df[master_merge_col]=df[master_merge_col].fillna(df[new_merge_col])
                df=df.drop(new_merge_col,axis=1)
            df.sort_values(by=master_merge_col,inplace=True)
            return {'df':df}
            #except:
            #    pass
    return {'df':None}

## FUNCTION - stack_csvs_into_df

In [29]:
def find_common_cols(list_of_dfs):
    common_cols=[*list_of_dfs[0].columns]
    for df in list_of_dfs[1:]:
        common_cols=[*set(common_cols).intersection(set(df.columns))]
        #for col in df.columns:
         #   if col not in common_cols:
        #        common_cols.remove(col)
    return common_cols

In [5]:
def stack_csvs_into_df(files=None,header_row=None):
    '''Stack multiple csvs on top of each other in a dataframe'''
    print(files)
    #try:
    dfs=[]
    for f in files:
        dfs.append(pd.read_csv(f,header=header_row))
    common_cols=find_common_cols(dfs)
    master_df=pd.DataFrame(columns=common_cols)
    for df in dfs:
        master_df=pd.concat([master_df,df[common_cols]])
    master_df.drop_duplicates(inplace=True)
    return {"df":master_df}
    #except:
    #    pass
    return {"df":None}